# CAPTSTONE PROJECT 3:

### DATA CLEANING

***

##### CLEAN DATA:
* Import .csv file in local Data Folder as DataFrame: df
* Examine the contents of DataFrame and address any issues regard NaN or others empty values
* Save updated dataFrame as df_cleaned.csv

In [1]:
# Import necessary modules for DataFrame cleaning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Import data from local file
data_path = r"C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_3\Data\Monthly_Transportation_Statistics.csv"
df = pd.read_csv(data_path)

In [3]:
df.head()

,Index,Date,Air Safety - General Aviation Fatalities,Highway Fatalities Per 100 Million Vehicle Miles Traveled,Highway Fatalities,U.S. Airline Traffic - Total - Seasonally Adjusted,U.S. Airline Traffic - International - Seasonally Adjusted,U.S. Airline Traffic - Domestic - Seasonally Adjusted,Transit Ridership - Other Transit Modes - Adjusted,Transit Ridership - Fixed Route Bus - Adjusted,...,Heavy truck sales SAAR (millions),U.S. Airline Traffic - Total - Non Seasonally Adjusted,Light truck sales SAAR (millions),U.S. Airline Traffic - International - Non Seasonally Adjusted,Auto sales SAAR (millions),U.S. Airline Traffic - Domestic - Non Seasonally Adjusted,Transborder - Total North American Freight,Transborder - U.S. - Mexico Freight,U.S. marketing air carriers on-time performance (percent),Transborder - U.S. - Canada Freight
0,0,01/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,02/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,03/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,04/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,05/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(924, 136)

In [5]:
df.isna()

,Index,Date,Air Safety - General Aviation Fatalities,Highway Fatalities Per 100 Million Vehicle Miles Traveled,Highway Fatalities,U.S. Airline Traffic - Total - Seasonally Adjusted,U.S. Airline Traffic - International - Seasonally Adjusted,U.S. Airline Traffic - Domestic - Seasonally Adjusted,Transit Ridership - Other Transit Modes - Adjusted,Transit Ridership - Fixed Route Bus - Adjusted,...,Heavy truck sales SAAR (millions),U.S. Airline Traffic - Total - Non Seasonally Adjusted,Light truck sales SAAR (millions),U.S. Airline Traffic - International - Non Seasonally Adjusted,Auto sales SAAR (millions),U.S. Airline Traffic - Domestic - Non Seasonally Adjusted,Transborder - Total North American Freight,Transborder - U.S. - Mexico Freight,U.S. marketing air carriers on-time performance (percent),Transborder - U.S. - Canada Freight
0,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,False,False,False,True,True,True,True,True,True,True,...,False,True,False,True,False,True,True,True,False,True
920,False,False,False,True,True,True,True,True,True,True,...,False,True,False,True,False,True,True,True,False,True
921,False,False,False,True,True,True,True,True,True,True,...,False,True,False,True,False,True,True,True,True,True
922,False,False,False,True,True,True,True,True,True,True,...,False,True,False,True,False,True,True,True,True,True


In [6]:
df.dtypes

Index                                                          int64
Date                                                          object
Air Safety - General Aviation Fatalities                     float64
Highway Fatalities Per 100 Million Vehicle Miles Traveled    float64
Highway Fatalities                                           float64
                                                              ...   
U.S. Airline Traffic - Domestic - Non Seasonally Adjusted    float64
Transborder - Total North American Freight                   float64
Transborder - U.S. - Mexico Freight                          float64
U.S. marketing air carriers on-time performance (percent)    float64
Transborder - U.S. - Canada Freight                          float64
Length: 136, dtype: object

In [7]:
df.describe()

,Index,Air Safety - General Aviation Fatalities,Highway Fatalities Per 100 Million Vehicle Miles Traveled,Highway Fatalities,U.S. Airline Traffic - Total - Seasonally Adjusted,U.S. Airline Traffic - International - Seasonally Adjusted,U.S. Airline Traffic - Domestic - Seasonally Adjusted,Transit Ridership - Other Transit Modes - Adjusted,Transit Ridership - Fixed Route Bus - Adjusted,Transit Ridership - Urban Rail - Adjusted,...,Heavy truck sales SAAR (millions),U.S. Airline Traffic - Total - Non Seasonally Adjusted,Light truck sales SAAR (millions),U.S. Airline Traffic - International - Non Seasonally Adjusted,Auto sales SAAR (millions),U.S. Airline Traffic - Domestic - Non Seasonally Adjusted,Transborder - Total North American Freight,Transborder - U.S. - Mexico Freight,U.S. marketing air carriers on-time performance (percent),Transborder - U.S. - Canada Freight
count,924.000000,509.000000,62.000000,55.000000,6.700000e+01,6.700000e+01,6.700000e+01,2.520000e+02,2.520000e+02,2.520000e+02,...,683.000000,6.700000e+01,6.830000e+02,6.700000e+01,6.830000e+02,6.700000e+01,2.030000e+02,2.030000e+02,69.000000,2.030000e+02
mean,461.500000,62.280943,1.184355,8914.490909,6.256746e+07,7.254478e+06,5.531299e+07,1.479240e+07,3.964658e+08,3.433674e+08,...,361298.682284,6.247284e+07,6.080971e+06,7.257015e+06,8.157646e+06,5.521582e+07,9.118179e+10,4.188672e+10,0.798551,4.929506e+10
std,266.880123,45.413863,0.117314,1063.909681,1.976432e+07,2.964934e+06,1.690153e+07,3.016048e+06,8.023240e+07,8.191928e+07,...,93852.969406,2.030572e+07,3.302128e+06,3.087197e+06,2.133211e+06,1.736053e+07,1.692174e+10,1.074877e+10,0.062996,7.156351e+09
min,0.000000,0.000000,0.980000,6726.000000,3.010000e+06,1.400000e+05,2.870000e+06,4.303718e+06,1.110446e+08,4.295134e+07,...,159000.000000,3.010000e+06,9.880000e+05,1.400000e+05,1.898000e+06,2.880000e+06,5.426452e+10,2.164871e+10,0.600000,2.976629e+10
25%,230.750000,33.000000,1.102500,8239.500000,6.146500e+07,5.565000e+06,5.566000e+07,1.264651e+07,3.921185e+08,2.998691e+08,...,297000.000000,5.822000e+07,3.192500e+06,5.545000e+06,7.356500e+06,5.332500e+07,8.034844e+10,3.254186e+10,0.800000,4.554671e+10
50%,461.500000,52.000000,1.170000,8975.000000,7.072000e+07,9.020000e+06,6.206000e+07,1.520069e+07,4.183203e+08,3.722867e+08,...,353000.000000,7.034000e+07,5.765000e+06,8.600000e+06,8.286000e+06,6.128000e+07,9.227839e+10,4.301536e+10,0.800000,4.890704e+10
75%,692.250000,78.000000,1.245000,9524.500000,7.459000e+07,9.320000e+06,6.528000e+07,1.700526e+07,4.428878e+08,4.011837e+08,...,432000.000000,7.464000e+07,8.670000e+06,9.125000e+06,9.610500e+06,6.669000e+07,9.995265e+10,4.859893e+10,0.800000,5.308561e+10
max,923.000000,345.000000,1.440000,11750.000000,8.060000e+07,9.790000e+06,7.087000e+07,2.069144e+07,5.155401e+08,4.547436e+08,...,583000.000000,8.696000e+07,1.396100e+07,1.167000e+07,1.505100e+07,7.528000e+07,1.418502e+11,7.029840e+10,0.900000,7.334734e+10


***
It appears that all NaN values represent non-existance of value and isn't necessarily representing failure to input. so a simple .fillna(0.0) function will suffice for this data set.

In [8]:
df = df.fillna(0.0)

In [9]:
df

,Index,Date,Air Safety - General Aviation Fatalities,Highway Fatalities Per 100 Million Vehicle Miles Traveled,Highway Fatalities,U.S. Airline Traffic - Total - Seasonally Adjusted,U.S. Airline Traffic - International - Seasonally Adjusted,U.S. Airline Traffic - Domestic - Seasonally Adjusted,Transit Ridership - Other Transit Modes - Adjusted,Transit Ridership - Fixed Route Bus - Adjusted,...,Heavy truck sales SAAR (millions),U.S. Airline Traffic - Total - Non Seasonally Adjusted,Light truck sales SAAR (millions),U.S. Airline Traffic - International - Non Seasonally Adjusted,Auto sales SAAR (millions),U.S. Airline Traffic - Domestic - Non Seasonally Adjusted,Transborder - Total North American Freight,Transborder - U.S. - Mexico Freight,U.S. marketing air carriers on-time performance (percent),Transborder - U.S. - Canada Freight
0,0,01/01/1947 12:00:00 AM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,02/01/1947 12:00:00 AM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,03/01/1947 12:00:00 AM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,04/01/1947 12:00:00 AM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,05/01/1947 12:00:00 AM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,919,08/01/2023 12:00:00 AM,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,544000.0,0.0,12196000.0,0.0,3099000.0,0.0,0.0,0.0,0.8,0.0
920,920,09/01/2023 12:00:00 AM,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,501000.0,0.0,12438000.0,0.0,3220000.0,0.0,0.0,0.0,0.8,0.0
921,921,10/01/2023 12:00:00 AM,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,451000.0,0.0,12408000.0,0.0,3038000.0,0.0,0.0,0.0,0.0,0.0
922,922,11/01/2023 12:00:00 AM,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,478000.0,0.0,12329000.0,0.0,2991000.0,0.0,0.0,0.0,0.0,0.0


***
Export df. to df_cleaned.csv for use in future Jupyter Notebooks.

In [10]:
df.to_csv(r"C:\Users\micha\OneDrive\Documents\GitHub\SpringBoardDataScience\Capstone_Project_3\Data\df_cleaned.csv")